# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

На первом этапе импортируем необходимые библиотеки.

In [256]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

Далее подгрузим данные.

In [257]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

Для упрощения работы создадим функцию, которая будет принимать на вход датасет, на выходе предоставлять всю необходимую информацию по нему.

In [258]:
def info(df_number):
    
    print(df_number.info())
    print(df_number.shape)
    display(df_number.head(10))

Ознакомимся с каждым датасетом отдельно.

In [259]:
info(df1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
(100000, 5)


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [260]:
info(df2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
(100000, 5)


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [261]:
info(df3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
(100000, 5)


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


В данных нет пропуском, тип данных удовлетворителен, можно начать подготовку к работе.

Для начала удалим *id* скважин. При обучение модели эти данные будут мешать, а при необходимости их можно будет восстановить по изначальным датасетам и числовым индексам.

In [262]:
df1 = df1.drop(columns='id', axis=1)
display(df1)
df2 = df2.drop(columns='id', axis=1)
display(df2)
df3 = df3.drop(columns='id', axis=1)
display(df3)

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...
99995,0.971957,0.370953,6.075346,110.744026
99996,1.392429,-0.382606,1.273912,122.346843
99997,1.029585,0.018787,-1.348308,64.375443
99998,0.998163,-0.528582,1.583869,74.040764


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...
99995,9.535637,-6.878139,1.998296,53.906522
99996,-10.160631,-12.558096,5.005581,137.945408
99997,-7.378891,-3.084104,4.998651,137.945408
99998,0.665714,-6.152593,1.000146,30.132364


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...
99995,-1.777037,1.125220,6.263374,172.327046
99996,-1.261523,-0.894828,2.524545,138.748846
99997,-1.199934,-2.957637,5.219411,157.080080
99998,-2.419896,2.417221,-5.548444,51.795253


### Вывод

Необходимые библиотеки загружены, данные проверены - замечаний нет. Можно переходить ко второму этапу работы.

## Обучение и проверка модели

Подготовим две функции:

1. Первая будет получать на вход датасет и разделять его на тренировочную и валидационную выборки.

2. Вторая будет на вход получать выборки из первой функции, обучать модель, делать предсказания, рассчитывать средний запас сырья и RMSE модели. Функция будет возвращать: обученную модель, предсказания, информацию об результатах предсказания.

In [263]:
def train_valid(df):
    # функция разбивки датасетов на выборки
    
    features = df.drop(columns='product', axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    
    return features_train, target_train, features_valid, target_valid

In [264]:
def predict_model(f_t, t_t, f_v, t_v):
    #функция обучения модели
    
    model = LinearRegression()
    model.fit(f_t, t_t)
    predict = model.predict(f_v)
    mean_p = predict.mean()
    rmse = mean_squared_error(predict, t_v) ** 0.5
    list_info = [mean_p, rmse]
    
    print('Средний запас сырья:', mean_p)
    print('RMSE', rmse)
    
    return model, predict, list_info

Теперь прогоним через функции каждый датасет в отдельности, параллельно сформируем таблицу, с оценками предсказаний моделей.

In [265]:
info_all = []
features_t_1, target_t_1, features_v_1, target_v_1 = train_valid(df1)
model_1, predict_1, info_1 = predict_model(features_t_1, target_t_1, features_v_1, target_v_1)
info_all.append(info_1)

Средний запас сырья: 92.59256778438035
RMSE 37.5794217150813


In [266]:
features_t_2, target_t_2, features_v_2, target_v_2 = train_valid(df2)
model_2, predict_2, info_2 = predict_model(features_t_2, target_t_2, features_v_2, target_v_2)
info_all.append(info_2)

Средний запас сырья: 68.728546895446
RMSE 0.893099286775617


In [267]:
features_t_3, target_t_3, features_v_3, target_v_3 = train_valid(df3)
model_3, predict_3, info_3 = predict_model(features_t_3, target_t_3, features_v_3, target_v_3)
info_all.append(info_3)

Средний запас сырья: 94.96504596800489
RMSE 40.02970873393434


Ознакомимся с полученными средними предсказанными значениями объёмов скважин и оценкой предсказаний.

In [268]:
info_all = pd.DataFrame(data=info_all, columns=['mean', 'rmse'], index=['Скважина 1', 'Скважина 2', 'Скважина 3'])
info_all

,mean,rmse
Скважина 1,92.592568,37.579422
Скважина 2,68.728547,0.893099
Скважина 3,94.965046,40.029709


### Вывод

На данном этапе были подготовленны вспомагательные фукнции по разбивке датасетов на выборке и обучению модели. Далее, через функции были прогнанны все три датасета. В результате сформирована информационная таблица из которой видно, что:

1. По предсказаниям модели, наибольший средний объем скважин находится в третьем регионе.
2. Наилучшие предсказания модель сделала для второго региона.

## Подготовка к расчёту прибыли

Введем все необходимые константы из условий задачи.

In [269]:
count_oilwell = 500
develop_oilwell = 200
profit_for_oilwell = 450000
budget = 10 ** 10

Рассчитаем средний потребный объем скважин в регионе, для неубыточности компании.

In [270]:
mean_volume = budget/ (profit_for_oilwell * develop_oilwell)
print(f'Потребный средний объем скважины: {mean_volume:.3f}')

Потребный средний объем скважины: 111.111


Подготовим функцию расчёта прибыли, на вход функция получает предсказанные объемы скважин, и правильные ответы. Выводит на экран размер прибыли, а возвращает объект Series с предсказаниями, для дальнейшей работы.

In [271]:
def sum_profit(predict, target):
    pr = pd.Series(predict, index=target.index)
    profit = pr.sort_values(ascending=False).iloc[:200].sum()
    profit = (profit * profit_for_oilwell) - 10 ** 10
    print(f'Прибыль составит: {profit:.3f}')
    return pr

## Расчёт прибыли и рисков 

С помощью функии, из прошлой части работы, рассчитаем прибыль по скважинам для каждого региона.

In [272]:
pr1 = sum_profit(predict_1, target_v_1)

Прибыль составит: 3996048877.465


In [273]:
pr2 = sum_profit(predict_2, target_v_2)

Прибыль составит: 2485712051.974


In [274]:
pr3 = sum_profit(predict_3, target_v_3)

Прибыль составит: 3321754396.243


В данном формате расчётов, во всех регионах, компании дальная получить прибыль. Теперь проведем такой же рассчёт используя Bootstrap. Найдем доверительный интервал 95% и вероятность получить убыток в каждом регионе.

Для этого подготовим две фукнции.

In [275]:
def revenue(target, prob, count):
    #функция отбирает из 500 скважин 200 лучшие и по ним считаем прибыль\убыток
    probs_sorted = prob.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10**10

def sum_profit(target, prob):
    # функция выполняет bootstrap на 1000 выборок, считает доверительные интервалы и вероятность убытка.
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        # используются переменные от прошлой части работы (count_oilwell and develop_oilwell)
        target_subsample = target.sample(n=count_oilwell, replace=True, random_state=state)
        probs_subsample = prob[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample, develop_oilwell))

    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    mean = values.mean()
    nr = norm(values.mean(), values.std())
    percent = nr.cdf(0)
    
    print(f"Средняя выручка: {mean:.3f}")
    print(f"2.5%-квантиль: {lower:.3f}")
    print(f"97.5%-квантиль: {upper:.3f}")
    print(f"Вероятность убытка: {(percent * 100):.3f}%")

Разчитаем для каждого региона: среднюю выручку, доверительный 95% интервал и вероятность получить убыток.

In [276]:
sum_profit(target_v_1, pr1)

Средняя выручка: 425938526.911
2.5%-квантиль: -102090094.838
97.5%-квантиль: 947976353.358
Вероятность убытка: 6.225%


In [277]:
sum_profit(target_v_2, pr2)

Средняя выручка: 515222773.443
2.5%-квантиль: 68873225.371
97.5%-квантиль: 931547591.257
Вероятность убытка: 0.937%


In [278]:
sum_profit(target_v_3, pr3)

Средняя выручка: 435008362.783
2.5%-квантиль: -128880547.330
97.5%-квантиль: 969706954.180
Вероятность убытка: 6.206%


### Выводы

В данной части работы, была посчитана прибыль компании, для каждого региона, на основе обучении модели без технологии Bootstrap. В результате, получили, что в каждом регионе, компания получит прибыль.

Далее, была проделана таже работа, но с технологией Bootstrap. Так же были посчитаны доверительные интервалы и вероятности получения убытка.

Как видно из расчётов, только в регионе 2 в 95% доверительном интервале компания не получает убытки, а их вероятность равна 0,937%. В остальных регионах и выроятность заметно выше, и в 95% доверительном интервале есть убытки. Таким образом следует выбрать для разработки регион №2.